# Dask SLURMCluster to create concatenated Zarr-Store

In this notebook we use `SLURMCluster` from the `dask_jobqueue` package to create a large Zarr-Store with some predefined chunking strategy. With `SLURMCluster` we are able to scale the nice features of `xarray` and `Dask` across multiple workers/nodes of a HPC. We can then monitor the progress of our jobs using the informative `Dask` Dashboard (see representation below), which allows us to check quickly if the chunking pattern/cluster specification we have chosen is optimal.

![Dask Dashboard](../docs/images/dashboard_status.png "Dask Dashboard")

In order to run this notebook you may need to install and activate the project Python environment and install the ipykernel using the below commands.
1. Create a virtual environment in desired location (e.g. `$PERM`):

    ```bash
    module load python3
    mkdir -p $PERM/venvs
    cd $PERM/venvs
    python3 -m venv --system-site-packages ec_land_db
    source ec_land_db/bin/activate
    ```
2. Navigate back to project directory and pip install:

    ```bash
    pip install -e .
    ```
3. Install the ipykernel:

    ```bash
    ipython kernel install --user --name=ec_land_db
    ```

In [1]:
import os
import logging
from typing import Dict
import xarray as xr
import numpy as np
import yaml
os.environ['HDF5_USE_FILE_LOCKING']="FALSE"

# Create some helper functions for later
def reset_chunk_encoding(da: xr.Dataset) -> xr.Dataset:
    """resets previous chunk encoding on concatenated zarr stores

    :param da: Dataset
    :return: Dataset with reset chunk encoding
    """
    for x in da.coords:
        if "chunks" in da[x].encoding.keys():
            logging.info(f"deleting chunk encoding for {x}...")
            del da[x].encoding["chunks"]
        if da.coords[x].dtype == object:
            da[x].encoding.clear()

    for var in da:
        if "chunks" in da[var].encoding.keys():
            logging.info(f"deleting chunk encoding for {var}...")
            del da[var].encoding["chunks"]
        if da[x].dtype == object:
            da[x].encoding.clear()
    return da


def parse_yaml(yaml_path: str) -> Dict:
    """Helper function to open config yaml files

    :param yaml_path: path to config yaml
    :return: dictionary of meta-data for group name
    """
    with open(yaml_path, "r") as yaml_file:
        obs_dic = yaml.safe_load(yaml_file)
    return obs_dic

# Load config yaml and inspect contents
CONFIG_DIC = parse_yaml("../config.yaml")
CONFIG_DIC

{'expver': 'i6aj',
 'start_date': '20200201',
 'end_date': '20221201',
 'climfile': '/home/daep/projects/ec_land_db/ec_land_db/scratch/surfclim_399_l4',
 'output_dir': '/ec/res4/scratch/daep/ec_training_db_out',
 'grid': 'O400',
 'time_chunk_size': 10}

## Setting up cluster arguemnts

In the next cell we set the directive we want each Dask worker to use

In [2]:
from dask_jobqueue import SLURMCluster
from distributed import Client

cluster = SLURMCluster(
    cores=4,  # each worker will have 4 threads
    processes=1,
    
    memory="24GB",  # each worker will have 24GB memory
    # interface="ib0",  # makes sure workers can communicate with eachother, maybe not needed
    walltime="01:30:00",  # set walltime of Dask workers
    job_extra_directives=['--qos=nf',  # spin up each worker on "nf" queue
                          '--output=logfiles/dask.%j.out',
                          '--error=logfiles/dask.%j.out'], 
    job_script_prologue=['module load conda; conda activate ec_land_db; export HDF5_USE_FILE_LOCKING=FALSE'],  # ensure correct python environment used
)

In [3]:
# See job script that will be submitted by Dask cluster
print(cluster.job_script())

#!/usr/bin/env bash

#SBATCH -J dask-worker
#SBATCH -e /tmp/dask-worker-%J.err
#SBATCH -o /tmp/dask-worker-%J.out
#SBATCH -n 1
#SBATCH --cpus-per-task=4
#SBATCH --mem=23G
#SBATCH -t 01:30:00
#SBATCH --qos=nf
#SBATCH --output=logfiles/dask.%j.out
#SBATCH --error=logfiles/dask.%j.out
module load conda; conda activate ec_land_db; export HDF5_USE_FILE_LOCKING=FALSE
/usr/local/apps/python3/3.10.10-01/bin/python3.10 -m distributed.cli.dask_worker tcp://10.100.192.102:46345 --nthreads 4 --memory-limit 22.35GiB --name dummy-name --nanny --death-timeout 60s --local-directory $TMPDIR --interface ib0



In [ ]:
# This command shutsdown our client and Dask cluster 
# ** DONT FORGET TO USE WHEN FINISED! **
client.shutdown()

## Launching the cluster

No we launch the cluster and can chose the amount of jobs we want to run

In [5]:
# scale cluse to desired number of workers, here 10
cluster.scale(jobs=10)

In [7]:
# Watch the dask-workers spinning up
# Wait until dask works have started running
!squeue -u $USER

    JOBID       NAME  USER   QOS    STATE       TIME TIME_LIMIT NODES      FEATURES NODELIST(REASON)
 30417803 dask-worke  daep    nf  RUNNING       0:02    1:30:00     1        (null) ac6-192
 30417804 dask-worke  daep    nf  RUNNING       0:02    1:30:00     1        (null) ac6-104
 30417806 dask-worke  daep    nf  RUNNING       0:02    1:30:00     1        (null) ac6-108
 30417809 dask-worke  daep    nf  RUNNING       0:02    1:30:00     1        (null) ac6-109
 30417813 dask-worke  daep    nf  RUNNING       0:02    1:30:00     1        (null) ac6-114
 30417816 dask-worke  daep    nf  RUNNING       0:02    1:30:00     1        (null) ac6-121
 30417818 dask-worke  daep    nf  RUNNING       0:02    1:30:00     1        (null) ac6-122
 30417819 dask-worke  daep    nf  RUNNING       0:02    1:30:00     1        (null) ac6-124
 30417820 dask-worke  daep    nf  RUNNING       0:02    1:30:00     1        (null) ac6-134
 30417821 dask-worke  daep    nf  RUNNING       0:02    1:30:00     1  

Once the workers are up and running we can connect the client to the cluster and view the dashboard. If on the VDI we can simply click on the link after "Dashboard: ...". 

If connecting using `tsh` we can instead open a new local terminal run: `ssh hpc-login -L 8787:aa6-204.bullx:8787`, replacing `aa6-204` with the host name you are connected to in your local JupyterLab or VSCode session and then navigate to `http://localhost:8787/status` in your web browser.

In [8]:
# Connect our local client to the dask-workers we have spun up
client = Client(cluster)
client

<Client: 'tcp://10.100.192.102:46345' processes=10 threads=40, memory=223.50 GiB>

## Opening the data

Firstly we open up the monthly ec-land forcing which we have already grabed and processed according to the `config.yaml` and project `README.md`. We use the `open_mfdataset` function from `xarray`.

In [9]:
ds_met = xr.open_mfdataset(f"{CONFIG_DIC['output_dir']}/ecland_era5forcing_2019*.zarr", engine="zarr", parallel=True,)

# rename the data variables to prefix that they are meteorological forcing
ds_met = ds_met.rename({var: f"met_{var.lower()}" for var in ds_met.data_vars})

ds_met

<xarray.Dataset>
Dimensions:     (time: 1460, x: 187070)
Coordinates:
    lat         (x) float32 dask.array<chunksize=(187070,), meta=np.ndarray>
    lon         (x) float32 dask.array<chunksize=(187070,), meta=np.ndarray>
  * time        (time) datetime64[ns] 2019-01-01 ... 2019-12-31T18:00:00
  * x           (x) int32 2192 2193 2194 2195 ... 654397 654398 654399 654400
Data variables:
    met_ctpf    (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    met_lwdown  (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    met_psurf   (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    met_qair    (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    met_rainf   (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    met_swdown  (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    met_snowf   (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    met_tair    (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    met_wind_e  (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    met_wind_n  (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
Attributes:
    CONVERTED:     from grib files with grib_api
    GRID_POINTS:  gaussian grid 
    SOURCE:       ECMWF

Next we open the ec-land model output which has been downloaded from Mars and processed according to the `config.yaml`.

In [10]:
ds_mod = xr.open_mfdataset(f"{CONFIG_DIC['output_dir']}/{CONFIG_DIC['expver']}_2019*.zarr", engine="zarr", parallel=True,)
for var in list(ds_mod.data_vars.keys()):  # rename the data variables to remove any leading 0's
    if var[0] == "0":
        ds_mod = ds_mod.rename({var: var[1:]})
ds_mod

<xarray.Dataset>
Dimensions:      (time: 1460, x: 187070)
Coordinates:
    lat          (x) float64 dask.array<chunksize=(187070,), meta=np.ndarray>
    lon          (x) float64 dask.array<chunksize=(187070,), meta=np.ndarray>
  * time         (time) datetime64[ns] 2019-01-01 ... 2019-12-31T18:00:00
  * x            (x) int32 2192 2193 2194 2195 ... 654397 654398 654399 654400
Data variables: (12/42)
    aco2gpp      (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    clim_CLAKE   (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    clim_Ctype   (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    clim_cu      (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    clim_cvh     (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    clim_cvl     (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    ...           ...
    swvl3        (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    swvl4        (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    t2m          (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    tsn          (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    tsn_l1       (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    tsn_l2       (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
Attributes:
    Conventions:             CF-1.7
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_edition:            1
    GRIB_subCentre:          0
    history:                 2023-10-27T11:30 GRIB to CDM+CF via cfgrib-0.9.1...
    institution:             European Centre for Medium-Range Weather Forecasts

## Merging datasets and writing to new Zarr-Store

now we can merge the datasets of met-forcing and ec-land output and save to a consolidated Zarr-Store with a predefined chunking strategy.

In [11]:
ds_train = reset_chunk_encoding(xr.merge([ds_met, ds_mod.drop(["lat", "lon"])]))
ds_train

<xarray.Dataset>
Dimensions:      (time: 1460, x: 187070)
Coordinates:
    lat          (x) float32 dask.array<chunksize=(187070,), meta=np.ndarray>
    lon          (x) float32 dask.array<chunksize=(187070,), meta=np.ndarray>
  * time         (time) datetime64[ns] 2019-01-01 ... 2019-12-31T18:00:00
  * x            (x) int32 2192 2193 2194 2195 ... 654397 654398 654399 654400
Data variables: (12/52)
    met_ctpf     (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    met_lwdown   (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    met_psurf    (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    met_qair     (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    met_rainf    (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    met_swdown   (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    ...           ...
    swvl3        (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    swvl4        (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    t2m          (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    tsn          (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    tsn_l1       (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    tsn_l2       (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
Attributes:
    CONVERTED:     from grib files with grib_api
    GRID_POINTS:  gaussian grid 
    SOURCE:       ECMWF

In [12]:
# Choose exactly which variables we want in our training database...
ml_database_varlst = [
 'met_ctpf',
 'met_lwdown',
 'met_psurf',
 'met_qair',
 'met_rainf',
 'met_swdown',
 'met_snowf',
 'met_tair',
 'met_wind_e',
 'met_wind_n',
 'clim_CLAKE',
 'clim_Ctype',
 'clim_cu',
 'clim_cvh',
 'clim_cvl',
 'clim_geopot',
 'clim_sdfor',
 'clim_sdor',
 'clim_sotype',
 'clim_tvh',
 'clim_tvl',
 'clim_z0m',
 'lai_hv',
 'lai_lv',
 'aco2gpp',
 # 'd2m',
 # 'dis',
 # 'e',
 # 'fal',
 # 'fldsto',
 'rsn',
 'rsn_l1',
 # 'rsn_l2',
 'sd',
 'sd_l1',
 # 'sd_l2',
 'skt',
 'snowc',
 'sro',
 'ssro',
 'stl1',
 'stl2',
 'stl3',
 # 'stl4',
 'swvl1',
 'swvl2',
 'swvl3',
 # 'swvl4',
 # 't2m',
 # 'tsn',
 # 'tsn_l1',
 # 'tsn_l2'
]

Select a reduced set of features to write and check size of dataset

In [13]:
ds_train = ds_train[ml_database_varlst]
print(f"size of dataset = {ds_train.nbytes*1e-9:.2f} GB")
ds_train

size of dataset = 42.61 GB


<xarray.Dataset>
Dimensions:      (time: 1460, x: 187070)
Coordinates:
    lat          (x) float32 dask.array<chunksize=(187070,), meta=np.ndarray>
    lon          (x) float32 dask.array<chunksize=(187070,), meta=np.ndarray>
  * time         (time) datetime64[ns] 2019-01-01 ... 2019-12-31T18:00:00
  * x            (x) int32 2192 2193 2194 2195 ... 654397 654398 654399 654400
Data variables: (12/39)
    met_ctpf     (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    met_lwdown   (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    met_psurf    (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    met_qair     (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    met_rainf    (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    met_swdown   (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    ...           ...
    stl1         (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    stl2         (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    stl3         (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    swvl1        (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    swvl2        (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
    swvl3        (time, x) float32 dask.array<chunksize=(124, 187070), meta=np.ndarray>
Attributes:
    CONVERTED:     from grib files with grib_api
    GRID_POINTS:  gaussian grid 
    SOURCE:       ECMWF

## Write consolidated data to Zarr-Store

Use chunk size defined in `config.yaml` and write data to Zarr-Store. If we have the Dask Dashboard running we can check this to ensure everything runs well and that we haven't specified a suboptimal chunking strategy.

In [14]:
%%time
ds_train.chunk({"time": CONFIG_DIC["time_chunk_size"]}).to_zarr(f"{CONFIG_DIC['output_dir']}/ecland_{CONFIG_DIC['expver']}_2019.zarr", consolidated=True)

CPU times: user 9.04 s, sys: 439 ms, total: 9.48 s
Wall time: 13.5 s


## Check that the data is there and as expected

We have written the data now and can inspect the dataset on disk

In [15]:
ds_test = xr.open_zarr(f"{CONFIG_DIC['output_dir']}/ecland_{CONFIG_DIC['expver']}_2019.zarr")
ds_test

<xarray.Dataset>
Dimensions:      (time: 1460, x: 187070)
Coordinates:
    lat          (x) float32 dask.array<chunksize=(187070,), meta=np.ndarray>
    lon          (x) float32 dask.array<chunksize=(187070,), meta=np.ndarray>
  * time         (time) datetime64[ns] 2019-01-01 ... 2019-12-31T18:00:00
  * x            (x) int32 2192 2193 2194 2195 ... 654397 654398 654399 654400
Data variables: (12/39)
    aco2gpp      (time, x) float32 dask.array<chunksize=(10, 187070), meta=np.ndarray>
    clim_CLAKE   (time, x) float32 dask.array<chunksize=(10, 187070), meta=np.ndarray>
    clim_Ctype   (time, x) float32 dask.array<chunksize=(10, 187070), meta=np.ndarray>
    clim_cu      (time, x) float32 dask.array<chunksize=(10, 187070), meta=np.ndarray>
    clim_cvh     (time, x) float32 dask.array<chunksize=(10, 187070), meta=np.ndarray>
    clim_cvl     (time, x) float32 dask.array<chunksize=(10, 187070), meta=np.ndarray>
    ...           ...
    stl1         (time, x) float32 dask.array<chunksize=(10, 187070), meta=np.ndarray>
    stl2         (time, x) float32 dask.array<chunksize=(10, 187070), meta=np.ndarray>
    stl3         (time, x) float32 dask.array<chunksize=(10, 187070), meta=np.ndarray>
    swvl1        (time, x) float32 dask.array<chunksize=(10, 187070), meta=np.ndarray>
    swvl2        (time, x) float32 dask.array<chunksize=(10, 187070), meta=np.ndarray>
    swvl3        (time, x) float32 dask.array<chunksize=(10, 187070), meta=np.ndarray>
Attributes:
    CONVERTED:     from grib files with grib_api
    GRID_POINTS:  gaussian grid 
    SOURCE:       ECMWF